In [2]:
from nltk.corpus import gutenberg

In [3]:
import nltk;
nltk.corpus.gutenberg.fileids()


['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [23]:
a= nltk.corpus.gutenberg.fileids()

In [24]:
print(gutenberg.raw('austen-emma.txt')[0:1000])

[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died too long ago for her to have more than an indistinct
remembrance of her caresses; and her place had been supplied
by an excellent woman as governess, who had fallen little short
of a mother in affection.

Sixteen years had Miss Taylor been in Mr. Woodhouse's family,
less as a governess than a friend, very fond of both daughters,
but particularly of Emma.  Between _them_ it was more the intimacy
of sisters.  Even before Miss Taylor had ceased to hold the nominal
office of governess, the mildness o

In [15]:
from nltk.corpus import gutenberg   # Docs from project gutenberg.org
import re
from nltk.corpus import stopwords 
files_en = gutenberg.fileids()      # Get file ids
doc_en1 = gutenberg.open('edgeworth-parents.txt').read()
doc_en2 = gutenberg.open('milton-paradise.txt').read()
doc_en3 = gutenberg.open('shakespeare-macbeth.txt').read()
tokens_en1 = re.sub('[^A-Za-z]', ' ', doc_en1)
tokens_en2 = re.sub('[^A-Za-z]', ' ', doc_en2)
tokens_en3 = re.sub('[^A-Za-z]', ' ', doc_en3)
#nltk.download('gutenberg')
tokens_en_lower1 = tokens_en1.lower()
tokens_en_lower2 = tokens_en2.lower()
tokens_en_lower3 = tokens_en3.lower()
from nltk.tokenize import word_tokenize
tokens_en_lower1 = word_tokenize(tokens_en_lower1)
tokens_en_lower2 = word_tokenize(tokens_en_lower2)
tokens_en_lower3 = word_tokenize(tokens_en_lower3)

filtered_words1 = [word for word in tokens_en_lower1 if word not in stopwords.words('english')]
filtered_words2 = [word for word in tokens_en_lower2 if word not in stopwords.words('english')]
filtered_words3 = [word for word in tokens_en_lower3 if word not in stopwords.words('english')]
en1 = nltk.Text(filtered_words1)
en2 = nltk.Text(filtered_words2)
en3 = nltk.Text(filtered_words3)

print(len(en1.tokens))       # returns number of tokens (document length)
print(len(set(en1.tokens)))  # returns number of unique tokens
print(len(en2.tokens)) 
len(filtered_words1)
len(filtered_words2)
len(filtered_words3)

78118
8169
45550


10058

In [26]:
len(filtered_words1)

78118

In [16]:
def sampling_6(data, n=10):
    data=copy.deepcopy(data);
    sample=[];
    for i in range(n):
        idx = random.randint(0,len(data)-1);
        a =sample.append(data.pop(idx));
    return sample

In [17]:
import random
import copy
sampledData1=[]
sampledData2=[]
sampledData3=[]
for i in range(200): 
    a=sampling_6(filtered_words1, n=150)
    b=sampling_6(filtered_words2, n=150)
    c=sampling_6(filtered_words3, n=150)
    sampledData1.append(a)
    sampledData2.append(b)
    sampledData3.append(c)
    

In [36]:
import pandas as pd
dataframe1 = pd.DataFrame(sampledData1)
dataframe2 = pd.DataFrame(sampledData2)
dataframe3 = pd.DataFrame(sampledData3)


In [19]:
dataframe3

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,subiect,sent,play,longer,times,receyue,bad,night,macb,art,...,stirre,got,thy,vs,vpon,hath,restlesse,doct,fill,hostesse
1,throw,substances,shall,bosome,caru,tydings,yet,instant,truth,macb,...,macb,lapt,good,owle,sword,worke,childe,breake,good,vs
2,lord,hands,lime,macduffe,man,owne,prima,fill,noble,bad,...,thou,noblenesse,hangmans,told,forgiue,performe,filthy,would,wife,castles
3,painting,banquo,burne,man,come,macb,awake,rise,like,vpon,...,affraid,nature,macduff,goe,let,transported,approach,scena,home,pray
4,heere,hec,enough,go,deepe,th,ouercome,damn,rather,traueller,...,macb,hidden,vnkle,sleepe,forc,goodnight,top,st,surfeted,comes
5,vpon,wee,macduffe,filthie,hoa,execution,taste,mal,prouokes,dudgeon,...,royall,makes,witnesse,cleare,dayes,men,charge,pyramids,pareill,actus
6,broke,shew,griefe,vnder,wearie,sweares,let,lady,therefore,dare,...,thought,st,filthie,like,king,within,craue,highnesse,moone,king
7,later,forke,spirits,owne,thus,affear,digestion,exeunt,hearts,wilt,...,night,madame,death,high,graue,rung,king,cap,doe,cheere
8,keepe,vnkle,wee,ribbes,almost,hang,enter,fry,hee,good,...,th,gentleman,skinne,scap,say,duncan,death,lesse,sigh,sundry
9,take,vnmake,behold,come,doome,cursed,doct,let,fall,noble,...,vs,sir,added,th,see,rather,doct,well,giue,healing


In [37]:
dataframe1 =dataframe1.apply(" ".join, axis=1)
dataframe2 =dataframe2.apply(" ".join, axis=1)
dataframe3 =dataframe3.apply(" ".join, axis=1)

In [11]:
dataframe3

0      prai father hither thy longer ile tooke heeles...
1      eyes beare ophe commend ham king good deare th...
2      theame like thou doomesday saint illium hamlet...
3      haue dreadfull steele king friend valentine sh...
4      effect towring bee fantasie knowes lord matter...
5      affliction prolongs time prate quicke ham ham ...
6      turn sigh maiestie vp mother winds freely giue...
7      ham father though earth must ham let drinke vo...
8      cautell nunnerie pause polon therefore flouris...
9      would againe set yet stoupe bespeake lazar mig...
10     shall acquire fauour target desperate ham shol...
11     know ground foot months downe attendants king ...
12     womans intreate clay sent length hatch friend ...
13     shewes antike talk selfe lacke made hor finde ...
14     supposall burst england would well deede insta...
15     marke forehead reuenge haue tell damned true h...
16     father husbands murthers ham hatchment hamlet ...
17     nay trauaile may masters

In [48]:
dataframe1 = pd.DataFrame(dataframe1)
dataframe2 = pd.DataFrame(dataframe2)
dataframe3 = pd.DataFrame(dataframe3)
dataframe1.columns = ['Sample']
dataframe2.columns = ['Sample']
dataframe3.columns = ['Sample']

In [24]:
c = pd.DataFrame({'author':[],'Genre':[]})

In [49]:
dataframe1

,Sample
0,trovano business aunt belief sir become poll w...
1,nothing brought earth many man time crown beha...
2,call mr five evening found book much marianne ...
3,understanding happen many go otherwise place c...
4,sir without carefully fellow go everybody gard...
5,lengthen silver took knees macaroon yesterday ...
6,tongue undrawing good would restless little bu...
7,night violetta done cart public cried plums ag...
8,stitch pains maurice exclaimed provide soon bi...
9,sudden matter piedro left j mr came droll land...


In [60]:
frames1 =[dataframe1,c]
frames2 =[dataframe2,c]
frames3 =[dataframe3,c]

In [61]:
result1 = pd.concat(frames1)
result2 = pd.concat(frames2)
result3 = pd.concat(frames3)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the war

In [67]:
result1 = result1[['Sample','author','Genre']]
result2 = result1[['Sample','author','Genre']]
result3 = result1[['Sample','author','Genre']]

In [69]:
for i in range(200):
    result1['Genre'] ="Fiction"
    result1['author'] =0
    result2['Genre'] ="Poetry"
    result2['author'] =1
    result3['Genre'] ="Tragedy"
    result3['author'] =2

In [88]:
frames = [result1, result2, result3]

In [89]:
result = pd.concat(frames)

In [103]:
result.sample(frac=1)

,Sample,author,Genre
177,bank silence fine bold people leonora come pla...,0,Fiction
163,joy cecilia hundred helped guineas rival must ...,0,Fiction
198,hear ease cecilia treasure though edmund five ...,2,Tragedy
77,speak coming grow indeed dame reminded questio...,1,Poetry
199,little activity spark basket house little brac...,1,Poetry
113,earned boy six rory half guinea said ever view...,2,Tragedy
62,gave booty three tarlton maurice shelves needl...,1,Poetry
31,moment better leading dream stood shook pray o...,1,Poetry
10,dependent spoke countenance sir ashamed wheel ...,0,Fiction
147,others letter captain take propriety plants th...,0,Fiction


In [94]:
    from sklearn.feature_extraction.text import CountVectorizer
matrixBoW = CountVectorizer(max_features=1000)
XBoW = matrixBoW.fit_transform(result['Sample']).todense()
XBoW= pd.DataFrame(XBoW, columns= matrixBoW.get_feature_names())

In [102]:
XBoW

,abbey,able,accident,account,added,admired,advantage,advice,afraid,afterwards,...,would,write,wrong,ye,year,years,yes,yesterday,yet,young
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,1
6,1,0,0,0,0,0,0,0,0,0,...,2,0,0,0,1,0,0,0,1,0
7,0,0,0,0,1,0,0,0,1,0,...,3,0,0,0,0,0,0,0,0,1
8,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [96]:
from sklearn.feature_extraction.text import TfidfVectorizer
matrixTfidf = TfidfVectorizer()
XTfidf = matrixTfidf.fit_transform(result['Sample']).todense()
XTfidf= pd.DataFrame(XTfidf, columns= matrixTfidf.get_feature_names())

In [104]:

XTfidf

,abashed,abated,abbey,abilities,able,abroad,abruptly,absence,absolutely,accept,...,yield,yielded,yields,yon,yonder,young,younger,youngsters,youth,zooks
0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
1,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
2,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
3,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.097867,0.0,0.000000,0.0
4,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
5,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.052791,0.000000,0.0,0.000000,0.0
6,0.0,0.0,0.070937,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
7,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,...,0.094684,0.0,0.0,0.0,0.000000,0.054300,0.000000,0.0,0.000000,0.0
8,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.097347,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
9,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0


In [106]:

y = result['author']
# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(XBoW, y)

In [107]:
y = result['author']

# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(XTfidf, y)


In [108]:
# Naive Bayes 
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predict Class
y_pred = classifier.predict(X_test)

# Accuracy 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

In [109]:
accuracy

0.04

In [110]:
classifier

GaussianNB(priors=None, var_smoothing=1e-09)

In [111]:
y_pred

array([1, 1, 0, 0, 2, 2, 0, 0, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 2,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 2, 2, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 2, 1, 0, 0, 1,
       2, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 2, 2, 1, 2, 0, 1, 1,
       0, 1, 2, 0, 1, 0, 1, 0, 1, 2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 2, 0, 0, 0, 1, 2, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 2, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 2, 0, 2, 1, 0, 2, 1, 0, 0, 0, 1])

In [112]:
y_test

75     0
66     0
88     1
81     1
38     1
25     0
91     1
35     1
48     2
130    0
92     2
24     2
167    0
169    0
107    1
82     0
8      2
139    1
174    1
191    2
89     2
148    0
133    0
111    0
42     0
98     0
110    2
93     2
12     1
158    2
      ..
177    0
71     0
196    1
102    0
6      2
128    1
118    1
163    2
30     1
148    1
144    0
43     0
23     0
108    1
74     1
136    1
50     1
47     0
12     2
51     0
179    2
156    0
195    2
107    2
33     1
69     2
45     1
61     1
104    2
191    1
Name: author, Length: 150, dtype: int64

In [113]:
X_test

,abashed,abated,abbey,abilities,able,abroad,abruptly,absence,absolutely,accept,...,yield,yielded,yields,yon,yonder,young,younger,youngsters,youth,zooks
75,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
66,0.0,0.000000,0.000000,0.000000,0.073545,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
288,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
281,0.0,0.000000,0.000000,0.000000,0.075379,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
238,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.095493,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
25,0.0,0.000000,0.000000,0.000000,0.074424,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
291,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
235,0.0,0.124105,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
448,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.097812,0.0,0.000000,0.000000
130,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
